# Mga Mekanismo ng Atensyon at mga Transformer

Isang pangunahing kahinaan ng mga recurrent network ay ang lahat ng salita sa isang sequence ay may parehong epekto sa resulta. Nagdudulot ito ng hindi optimal na performance sa mga karaniwang LSTM encoder-decoder na modelo para sa mga sequence-to-sequence na gawain, tulad ng Named Entity Recognition at Machine Translation. Sa totoong buhay, may mga partikular na salita sa input sequence na mas may epekto sa mga output kaysa sa iba.

Isaalang-alang ang isang sequence-to-sequence na modelo, tulad ng machine translation. Ito ay ipinatutupad gamit ang dalawang recurrent network, kung saan ang isang network (**encoder**) ay nagko-collapse ng input sequence sa isang hidden state, at ang isa pa, ang **decoder**, ay nag-u-unroll ng hidden state na ito sa isinaling resulta. Ang problema sa ganitong paraan ay ang huling estado ng network ay nahihirapang maalala ang simula ng pangungusap, na nagdudulot ng mababang kalidad ng modelo sa mahahabang pangungusap.

Ang **Mekanismo ng Atensyon** ay nagbibigay ng paraan upang timbangin ang kontekstwal na epekto ng bawat input vector sa bawat output prediction ng RNN. Ang paraan ng pagpapatupad nito ay sa pamamagitan ng paglikha ng mga shortcut sa pagitan ng mga intermediate state ng input RNN at output RNN. Sa ganitong paraan, kapag gumagawa ng output symbol $y_t$, isasaalang-alang natin ang lahat ng input hidden states $h_i$, na may iba't ibang weight coefficients $\alpha_{t,i}$. 

![Larawan na nagpapakita ng encoder/decoder model na may additive attention layer](../../../../../translated_images/tl/encoder-decoder-attention.7a726296894fb567.webp)
*Ang encoder-decoder model na may additive attention mechanism mula sa [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf), na binanggit mula sa [blog post na ito](https://lilianweng.github.io/lil-log/2018/06/24/attention-attention.html)*

Ang attention matrix $\{\alpha_{i,j}\}$ ay kumakatawan sa antas kung saan ang ilang input na salita ay may papel sa pagbuo ng isang partikular na salita sa output sequence. Narito ang halimbawa ng ganitong matrix:

![Larawan na nagpapakita ng sample alignment na natagpuan ng RNNsearch-50, kinuha mula sa Bahdanau - arviz.org](../../../../../translated_images/tl/bahdanau-fig3.09ba2d37f202a6af.webp)

*Larawan mula sa [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf) (Fig.3)*

Ang mga mekanismo ng atensyon ay responsable sa karamihan ng kasalukuyan o malapit sa kasalukuyang state-of-the-art sa Natural Language Processing. Gayunpaman, ang pagdaragdag ng atensyon ay lubos na nagpapataas ng bilang ng mga parameter ng modelo na nagdulot ng mga isyu sa scaling sa mga RNN. Isang pangunahing limitasyon ng pag-scale ng RNN ay ang likas na katangian ng mga modelong ito na mahirap i-batch at i-parallelize ang training. Sa isang RNN, ang bawat elemento ng sequence ay kailangang iproseso nang sunud-sunod, na nangangahulugang hindi ito madaling ma-parallelize.

Ang pagsasama ng mga mekanismo ng atensyon sa limitasyong ito ay nagresulta sa paglikha ng mga Transformer Models na ngayon ay itinuturing na State of the Art, mula sa BERT hanggang sa OpenGPT3.

## Mga Transformer Model

Sa halip na ipasa ang konteksto ng bawat nakaraang prediksyon sa susunod na hakbang ng pagsusuri, ang **transformer models** ay gumagamit ng **positional encodings** at **attention** upang makuha ang konteksto ng isang input sa loob ng isang ibinigay na window ng teksto. Ang larawan sa ibaba ay nagpapakita kung paano nakukuha ng positional encodings na may atensyon ang konteksto sa loob ng isang window.

![Animated GIF na nagpapakita kung paano isinasagawa ang mga pagsusuri sa transformer models.](../../../../../lessons/5-NLP/18-Transformers/images/transformer-animated-explanation.gif) 

Dahil ang bawat input na posisyon ay independiyenteng naiuugnay sa bawat output na posisyon, ang mga transformer ay mas mahusay sa pag-parallelize kumpara sa mga RNN, na nagbibigay-daan sa mas malalaking at mas expressive na mga modelo ng wika. Ang bawat attention head ay maaaring gamitin upang matutunan ang iba't ibang relasyon sa pagitan ng mga salita na nagpapabuti sa mga gawain sa Natural Language Processing.

## Pagbuo ng Simpleng Transformer Model

Ang Keras ay walang built-in na Transformer layer, ngunit maaari tayong gumawa ng sarili natin. Tulad ng dati, magpo-focus tayo sa text classification ng AG News dataset, ngunit mahalagang banggitin na ang mga Transformer models ay nagpapakita ng pinakamahusay na resulta sa mas mahihirap na gawain sa NLP.


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

Ang mga bagong layer sa Keras ay dapat mag-subclass sa klase na `Layer`, at mag-implement ng `call` na method. Magsimula tayo sa **Positional Embedding** na layer. Gagamit tayo ng [ilang code mula sa opisyal na dokumentasyon ng Keras](https://keras.io/examples/nlp/text_classification_with_transformer/). Ipagpapalagay natin na ang lahat ng input sequences ay pinapad sa haba na `maxlen`.


In [2]:
class TokenAndPositionEmbedding(keras.layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = keras.layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = keras.layers.Embedding(input_dim=maxlen, output_dim=embed_dim)
        self.maxlen = maxlen

    def call(self, x):
        maxlen = self.maxlen
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x+positions

Ngayon, ipapatupad natin ang transformer block. Tatanggapin nito ang output ng naunang tinukoy na embedding layer:


In [3]:
class TransformerBlock(keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim, name='attn')
        self.ffn = keras.Sequential(
            [keras.layers.Dense(ff_dim, activation="relu"), keras.layers.Dense(embed_dim),]
        )
        self.layernorm1 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = keras.layers.Dropout(rate)
        self.dropout2 = keras.layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

Ngayon, handa na tayong i-define ang kumpletong modelo ng transformer:


In [4]:
embed_dim = 32  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer
maxlen = 256
vocab_size = 20000

model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_sequence_length=maxlen, input_shape=(1,)),
    TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim),
    TransformerBlock(embed_dim, num_heads, ff_dim),
    keras.layers.GlobalAveragePooling1D(),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(20, activation="relu"),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(4, activation="softmax")
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_vectorization (TextVect (None, 256)               0         
_________________________________________________________________
token_and_position_embedding (None, 256, 32)           648192    
_________________________________________________________________
transformer_block (Transform (None, 256, 32)           10656     
_________________________________________________________________
global_average_pooling1d (Gl (None, 32)                0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 20)                660       
_________________________________________________________________
dropout_3 (Dropout)          (None, 20)               

In [5]:
print('Training tokenizer')
model.layers[0].adapt(ds_train.map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

Training tokenizer
938/938 [==============================] - 45s 39ms/step - loss: 0.4978 - acc: 0.8068 - val_loss: 0.2808 - val_acc: 0.9124


## Mga Modelong Transformer ng BERT

**BERT** (Bidirectional Encoder Representations from Transformers) ay isang napakalaking multi-layer transformer network na may 12 layers para sa *BERT-base*, at 24 para sa *BERT-large*. Ang modelo ay unang sinanay gamit ang malaking corpus ng text data (WikiPedia + mga libro) gamit ang unsupervised training (pagpapredikta ng mga nakatagong salita sa isang pangungusap). Sa panahon ng pre-training, ang modelo ay nakakapulot ng mataas na antas ng pag-unawa sa wika na maaaring magamit sa iba pang mga dataset sa pamamagitan ng fine tuning. Ang prosesong ito ay tinatawag na **transfer learning**.

![larawan mula sa http://jalammar.github.io/illustrated-bert/](../../../../../translated_images/tl/jalammarBERT-language-modeling-masked-lm.34f113ea5fec4362.webp)

Maraming mga bersyon ng Transformer architectures kabilang ang BERT, DistilBERT, BigBird, OpenGPT3, at iba pa na maaaring i-fine tune.

Tingnan natin kung paano natin magagamit ang pre-trained na modelo ng BERT para sa paglutas ng ating tradisyunal na problema sa sequence classification. Hihiramin natin ang ideya at ilang code mula sa [opisyal na dokumentasyon](https://www.tensorflow.org/text/tutorials/classify_text_with_bert).

Upang mag-load ng pre-trained na mga modelo, gagamit tayo ng **Tensorflow hub**. Una, i-load natin ang BERT-specific vectorizer:


In [1]:
import tensorflow_text 
import tensorflow_hub as hub
vectorizer = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')

ModuleNotFoundError: No module named 'tensorflow_text'

In [7]:
vectorizer(['I love transformers'])

{'input_type_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
       dtype=int32)>,
 'input_word_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[  101,  1045,  2293, 19081,   102,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0, 

Mahalagang gamitin mo ang parehong vectorizer na ginamit sa orihinal na network na sinanay. Bukod dito, ang BERT vectorizer ay nagbabalik ng tatlong bahagi:
* `input_word_ids`, na isang sequence ng mga numero ng token para sa input na pangungusap
* `input_mask`, na nagpapakita kung aling bahagi ng sequence ang naglalaman ng aktwal na input, at kung alin ang padding. Katulad ito ng mask na ginawa ng `Masking` layer
* `input_type_ids` ay ginagamit para sa mga gawain sa language modeling, at nagbibigay-daan upang tukuyin ang dalawang input na pangungusap sa isang sequence.

Pagkatapos, maaari nating i-instantiate ang BERT feature extractor:


In [8]:
bert = hub.KerasLayer('https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/1')

In [9]:
z = bert(vectorizer(['I love transformers']))
for i,x in z.items():
    print(f"{i} -> { len(x) if isinstance(x, list) else x.shape }")

pooled_output -> (1, 128)
encoder_outputs -> 4
sequence_output -> (1, 128, 128)
default -> (1, 128)


Kaya, ang BERT layer ay nagbibigay ng ilang kapaki-pakinabang na resulta:
* Ang `pooled_output` ay resulta ng pag-average ng lahat ng tokens sa sequence. Maaari mo itong ituring bilang isang matalinong semantic embedding ng buong network. Katumbas ito ng output ng `GlobalAveragePooling1D` layer sa ating naunang modelo.
* Ang `sequence_output` ay ang output ng huling transformer layer (katumbas ng output ng `TransformerBlock` sa ating modelo sa itaas).
* Ang `encoder_outputs` ay ang mga output ng lahat ng transformer layers. Dahil nag-load tayo ng 4-layer BERT model (tulad ng maaari mong mahulaan mula sa pangalan, na naglalaman ng `4_H`), mayroon itong 4 na tensors. Ang huli ay pareho sa `sequence_output`.

Ngayon, magde-define tayo ng end-to-end classification model. Gagamit tayo ng *functional model definition*, kung saan ide-define natin ang model input, at pagkatapos ay magbibigay ng serye ng mga expression upang kalkulahin ang output nito. Gagawin din nating hindi-trainable ang mga BERT model weights, at ang final classifier lamang ang ating ita-train:


In [10]:
inp = keras.Input(shape=(),dtype=tf.string)
x = vectorizer(inp)
x = bert(x)
x = keras.layers.Dropout(0.1)(x['pooled_output'])
out = keras.layers.Dense(4,activation='softmax')(x)
model = keras.models.Model(inp,out)
bert.trainable = False
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'input_type_ids': ( 0           input_1[0][0]                    
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'pooled_output': (N 4782465     keras_layer[0][0]                
                                                                 keras_layer[0][1]                
                                                                 keras_layer[0][2]                
______________________________________________________________________________________________

In [11]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

938/938 [==============================] - 528s 559ms/step - loss: 0.8056 - acc: 0.6983 - val_loss: 0.5953 - val_acc: 0.7888


Kahit kaunti lang ang mga trainable na parameter, medyo mabagal ang proseso dahil mabigat ang computational load ng BERT feature extractor. Mukhang hindi natin naabot ang makatwirang accuracy, maaaring dahil sa kakulangan ng training o kakulangan ng mga parameter ng modelo.

Subukan nating i-unfreeze ang mga BERT weights at i-train ito rin. Kakailanganin nito ng napakaliit na learning rate, pati na rin mas maingat na training strategy gamit ang **warmup**, at ang **AdamW** optimizer. Gagamitin natin ang `tf-models-official` package para gumawa ng optimizer:


In [12]:
from official.nlp import optimization 
bert.trainable=True
model.summary()
epochs = 3
opt = optimization.create_optimizer(
    init_lr=3e-5,
    num_train_steps=epochs*len(ds_train),
    num_warmup_steps=0.1*epochs*len(ds_train),
    optimizer_type='adamw')

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer=opt)
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'input_type_ids': ( 0           input_1[0][0]                    
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'pooled_output': (N 4782465     keras_layer[0][0]                
                                                                 keras_layer[0][1]                
                                                                 keras_layer[0][2]                
______________________________________________________________________________________________

Tulad ng nakikita mo, medyo mabagal ang pagsasanay - pero maaari mong subukang mag-eksperimento at sanayin ang modelo nang ilang epochs (5-10) at tingnan kung makakakuha ka ng mas magandang resulta kumpara sa mga pamamaraang ginamit natin dati.

## Huggingface Transformers Library

Isa pang karaniwang (at medyo mas simple) paraan ng paggamit ng Transformer models ay ang [HuggingFace package](https://github.com/huggingface/), na nagbibigay ng mga simpleng building blocks para sa iba't ibang NLP tasks. Available ito para sa parehong Tensorflow at PyTorch, isa pang napakapopular na neural network framework.

> **Note**: Kung hindi ka interesado na makita kung paano gumagana ang Transformers library - maaari mong laktawan ang dulo ng notebook na ito, dahil wala kang makikitang malaki ang pagkakaiba mula sa ginawa natin sa itaas. Uulitin lang natin ang parehong mga hakbang ng pagsasanay sa BERT model gamit ang ibang library at mas malaking modelo. Dahil dito, ang proseso ay nangangailangan ng medyo mahabang pagsasanay, kaya maaaring gusto mo na lang tingnan ang code. 

Tingnan natin kung paano masosolusyunan ang ating problema gamit ang [Huggingface Transformers](http://huggingface.co).


Ang unang bagay na kailangan nating gawin ay pumili ng modelong gagamitin natin. Bukod sa ilang built-in na mga modelo, ang Huggingface ay mayroong [online na repositoryo ng mga modelo](https://huggingface.co/models), kung saan makakahanap ka ng mas maraming pre-trained na mga modelo mula sa komunidad. Ang lahat ng mga modelong ito ay maaaring i-load at gamitin sa pamamagitan lamang ng pagbibigay ng pangalan ng modelo. Ang lahat ng kinakailangang binary files para sa modelo ay awtomatikong mada-download.

Sa ilang pagkakataon, maaaring kailanganin mong i-load ang sarili mong mga modelo. Sa ganitong kaso, maaari mong tukuyin ang direktoryo na naglalaman ng lahat ng kaugnay na mga file, kabilang ang mga parameter para sa tokenizer, ang `config.json` na file na may mga parameter ng modelo, mga binary weights, at iba pa.

Mula sa pangalan ng modelo, maaari nating i-instantiate ang parehong modelo at ang tokenizer. Magsimula tayo sa tokenizer:


In [2]:
import transformers

# To load the model from Internet repository using model name. 
# Use this if you are running from your own copy of the notebooks
bert_model = 'bert-base-uncased' 

# To load the model from the directory on disk. Use this for Microsoft Learn module, because we have
# prepared all required files for you.
#bert_model = './bert'

tokenizer = transformers.BertTokenizer.from_pretrained(bert_model)

MAX_SEQ_LEN = 128
PAD_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
UNK_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.unk_token)

Ang `tokenizer` na object ay naglalaman ng `encode` na function na maaaring direktang gamitin upang i-encode ang teksto:


In [3]:
tokenizer.encode('Tensorflow is a great framework for NLP')

[101, 23435, 12314, 2003, 1037, 2307, 7705, 2005, 17953, 2361, 102]

Maaari rin nating gamitin ang tokenizer upang i-encode ang isang sequence sa paraang angkop para maipasa sa modelo, halimbawa, kabilang ang `token_ids`, `input_mask` na mga field, at iba pa. Maaari rin nating tukuyin na gusto natin ng Tensorflow tensors sa pamamagitan ng pagbibigay ng argumentong `return_tensors='tf'`:


In [4]:
tokenizer(['Hello, there'],return_tensors='tf')

{'input_ids': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[ 101, 7592, 1010, 2045,  102]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[0, 0, 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[1, 1, 1, 1, 1]], dtype=int32)>}

Sa ating kaso, gagamit tayo ng pre-trained na BERT model na tinatawag na `bert-base-uncased`. Ang *Uncased* ay nangangahulugang hindi sensitibo sa malalaking at maliliit na titik ang model.

Kapag tine-train ang model, kailangan nating magbigay ng tokenized na sequence bilang input, kaya magdidisenyo tayo ng data processing pipeline. Dahil ang `tokenizer.encode` ay isang Python function, gagamitin natin ang parehong paraan tulad ng sa nakaraang unit sa pamamagitan ng pagtawag dito gamit ang `py_function`:


In [31]:
def process(x):
    return tokenizer.encode(x.numpy().decode('utf-8'),return_tensors='tf',padding='max_length',max_length=MAX_SEQ_LEN,truncation=True)[0]

def process_fn(x):
    s = x['title']+' '+x['description']
    e = tf.py_function(process,inp=[s],Tout=(tf.int32))
    e.set_shape(MAX_SEQ_LEN)
    return e,x['label']

Ngayon maaari na nating i-load ang aktwal na modelo gamit ang `BertForSequenceClassification` na package. Tinitiyak nito na ang ating modelo ay mayroon nang kinakailangang arkitektura para sa klasipikasyon, kabilang ang panghuling classifier. Makakakita ka ng babala na nagsasaad na ang mga timbang ng panghuling classifier ay hindi na-initialize, at ang modelo ay mangangailangan ng pre-training - ayos lang iyon, dahil eksakto iyon ang gagawin natin!


In [32]:
model = transformers.TFBertForSequenceClassification.from_pretrained(bert_model,num_labels=4,output_attentions=False)

In [33]:
model.summary()

Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_75 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  3076      
Total params: 109,485,316
Trainable params: 109,485,316
Non-trainable params: 0
_________________________________________________________________


Tulad ng makikita mo mula sa `summary()`, ang modelo ay naglalaman ng halos 110 milyong mga parameter! Marahil, kung nais natin ng simpleng gawain ng klasipikasyon sa medyo maliit na dataset, hindi natin nais na i-train ang BERT base layer:


In [34]:
model.layers[0].trainable = False
model.summary()

Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_75 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  3076      
Total params: 109,485,316
Trainable params: 3,076
Non-trainable params: 109,482,240
_________________________________________________________________


Ngayon, handa na tayong magsimula sa pagsasanay!

> **Tandaan**: Ang pagsasanay ng full-scale na BERT model ay maaaring ubos ng maraming oras! Kaya't sasanayin lang natin ito para sa unang 32 batches. Ito ay para lamang ipakita kung paano isinasagawa ang pagsasanay ng modelo. Kung interesado kang subukan ang full-scale na pagsasanay - alisin lamang ang mga parameter na `steps_per_epoch` at `validation_steps`, at maghanda kang maghintay!


In [30]:
model.compile('adam','sparse_categorical_crossentropy',['acc'])
tf.get_logger().setLevel('ERROR')
model.fit(ds_train.map(process_fn).batch(32),validation_data=ds_test.map(process_fn).batch(32),steps_per_epoch=32,validation_steps=2)

32/32 [==============================] - 142s 4s/step - loss: 1.3896 - acc: 0.2500 - val_loss: 1.3863 - val_acc: 0.2480


Kung tataasan mo ang bilang ng mga iterations at maghihintay nang sapat na tagal, at magsasanay para sa ilang epochs, maaasahan mong ang BERT classification ang magbibigay ng pinakamataas na accuracy! Ito ay dahil ang BERT ay may mahusay nang pagkaunawa sa istruktura ng wika, at kailangan lang nating i-fine-tune ang panghuling classifier. Gayunpaman, dahil ang BERT ay isang malaking modelo, ang buong proseso ng pagsasanay ay tumatagal ng mahabang oras at nangangailangan ng malakas na computational power! (GPU, at mas mainam kung higit sa isa).

> **Note:** Sa ating halimbawa, gumagamit tayo ng isa sa pinakamaliit na pre-trained na BERT models. Mayroon ding mas malalaking modelo na malamang na magbigay ng mas magagandang resulta.


## Mga Mahalagang Punto

Sa yunit na ito, tinalakay natin ang mga pinakabagong arkitektura ng modelo na batay sa **transformers**. Ginamit natin ang mga ito para sa ating gawain sa text classification, ngunit katulad nito, maaaring gamitin ang mga BERT models para sa entity extraction, question answering, at iba pang mga gawain sa NLP.

Ang mga transformer models ang kasalukuyang nangunguna sa larangan ng NLP, at sa karamihan ng mga kaso, ito ang dapat na unang solusyon na subukan mo kapag gumagawa ng mga custom na solusyon sa NLP. Gayunpaman, ang pag-unawa sa mga pangunahing prinsipyo ng recurrent neural networks na tinalakay sa module na ito ay napakahalaga kung nais mong bumuo ng mga advanced na neural models.



---

**Paunawa**:  
Ang dokumentong ito ay isinalin gamit ang AI translation service na [Co-op Translator](https://github.com/Azure/co-op-translator). Bagama't sinisikap naming maging tumpak, pakitandaan na ang mga awtomatikong pagsasalin ay maaaring maglaman ng mga pagkakamali o hindi pagkakatugma. Ang orihinal na dokumento sa orihinal nitong wika ang dapat ituring na opisyal na sanggunian. Para sa mahalagang impormasyon, inirerekomenda ang propesyonal na pagsasalin ng tao. Hindi kami mananagot sa anumang hindi pagkakaunawaan o maling interpretasyon na maaaring magmula sa paggamit ng pagsasaling ito.
